In [3]:
pip install python-telegram-bot pymongo google-generativeai serpapi

Defaulting to user installation because normal site-packages is not writeable
  Using cached python_telegram_bot-21.10-py3-none-any.whl.metadata (17 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached serpapi-0.1.5-py2.py3-none-any.whl.metadata (10 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.24.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached google_api_python_client-2.160.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth-2.38.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Using cached proto_plus-1.26.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached googleapis_common_protos-1.66.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import logging
import os
from telegram import Update, ForceReply, ParseMode
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes, MessageHandler, filters
from pymongo import MongoClient
import google.generativeai as palm
from datetime import datetime
from serpapi import GoogleSearch  # For web search

# Initialize logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(_name_)

# Initialize environment variables (REPLACE WITH YOUR ACTUAL VALUES)
BOT_TOKEN = os.environ.get("BOT_TOKEN")  # Get from BotFather
MONGO_URI = os.environ.get("MONGO_URI")  # Your MongoDB connection string
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")  # Your Gemini API Key
SERPAPI_API_KEY = os.environ.get("SERPAPI_API_KEY") # Your SerpAPI key

# Initialize MongoDB client
mongo_client = MongoClient(MONGO_URI)
db = mongo_client["telegram_bot_db"]  # Replace with your DB name
users_collection = db["users"]
chats_collection = db["chats"]
files_collection = db["files"]

# Initialize Gemini API
palm.configure(api_key=GEMINI_API_KEY)

# Handlers
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.effective_user
    user_data = {
        "first_name": user.first_name,
        "username": user.username,
        "chat_id": update.effective_chat.id
    }
    users_collection.insert_one(user_data)
    await update.message.reply_text("Welcome!  Use /help for commands.")


async def chat(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_input = update.message.text
    prompt = f"User: {user_input}\nBot:"  # Clearer prompt formatting
    try:
        response = palm.generate_text(
            model="models/chat-bison-001",  # Or another suitable model
            prompt=prompt,
            temperature=0.7,
            max_output_tokens=800
        )
        bot_response = response.result

        chat_data = {
            "user_id": update.effective_user.id,
            "chat_id": update.effective_chat.id,
            "user_input": user_input,
            "bot_response": bot_response,
            "timestamp": datetime.now()
        }
        chats_collection.insert_one(chat_data)
        await update.message.reply_text(bot_response)

    except Exception as e:
        logger.error(f"Gemini API Error: {e}")
        await update.message.reply_text("Sorry, there was an error processing your request.")


async def analyze_image(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if update.message.photo:
        file_id = update.message.photo[-1].file_id
        new_file = await context.bot.get_file(file_id)
        file_path = await new_file.download_as_bytes()  # Download to memory

        # Placeholder for Image Analysis (REPLACE with actual analysis)
        try:
           
            analysis_result = "Image analysis is not yet implemented. (Placeholder)"  # Replace this

            file_metadata = {
                "user_id": update.effective_user.id,
                "chat_id": update.effective_chat.id,
                "file_name": f"{file_id}.jpg",  # Or determine actual file type
                "description": analysis_result,
                "timestamp": datetime.now()
            }
            files_collection.insert_one(file_metadata)
            await update.message.reply_text(analysis_result)
        except Exception as e:
            logger.error(f"Image Analysis Error: {e}")
            await update.message.reply_text("Error analyzing the image.")


async def web_search(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.message.text.replace("/websearch ", "")  # Extract the query
    try:
        search = GoogleSearch({
            "q": query,
            "api_key": SERPAPI_API_KEY
        })
        result = search.get_dict()
        organic_results = result.get("organic_results", [])[:3]  # Get top 3

        summary = f"Search results for: {query}\n\n"
        for res in organic_results:
            title = res.get("title", "No Title")
            link = res.get("link", "No Link")
            snippet = res.get("snippet", "No Snippet")
            summary += f"<b>{title}</b>\n{snippet}\n<a href='{link}'>{link}</a>\n\n"

        await update.message.reply_text(summary, parse_mode=ParseMode.HTML)
    except Exception as e:
        logger.error(f"Web Search Error: {e}")
        await update.message.reply_text("Error performing the web search.")



async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    help_text = """
    Available commands:
    /start - Start the bot
    /chat <message> - Chat with the bot (or just type your message)
    /websearch <query> - Search the web
    Send an image to analyze it.
    """
    await update.message.reply_text(help_text)

async def phone_number(update: Update, context: ContextTypes.DEFAULT_TYPE):
    contact = update.message.contact
    user_id = update.effective_user.id
    users_collection.update_one({"_id": user_id}, {"$set": {"phone_number": contact.phone_number}})
    await update.message.reply_text("Thank you for sharing your phone number!")

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    logger.error(f"Update {update} caused error {context.error}")
    await update.message.reply_text("An error occurred. Please try again later.")


async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text("Sorry, I didn't understand that command.")


async def chat_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await chat(update, context)

async def image_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await analyze_image(update, context)

async def contact_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await phone_number(update, context)


async def main():
    application = ApplicationBuilder().token(BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("help", help_command))
    application.add_handler(CommandHandler("websearch", web_search))
    application.add_handler(MessageHandler(filters.TEXT & (~filters.COMMAND), chat_message))
    application.add_handler(MessageHandler(filters.PHOTO, image_message))
    application.add_handler(MessageHandler(filters.CONTACT, contact_message))
    application.add_handler(MessageHandler(filters.COMMAND, unknown))  # Handle unknown commands

    application.add_error_handler(error_handler)

    await application.initialize()
    await application.start_polling()
    await application.idle()

if _name_ == "_main_":
    import asyncio
    asyncio.run(main())

ImportError: cannot import name 'ParseMode' from 'telegram' (C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\telegram\__init__.py)